## 웹브라우저와 상호 작용

### Webdriver

[chrome drivers](https://sites.google.com/a/chromium.org/chromedriver/downloads)

In [1]:
from pyautomate import web

In [2]:
web.setup_webdriver(2.32)

chromedriver 다운로드
다운로드 받은 파일 압축 해제 ... 완료


브라우저 열기

In [3]:
chrome = web.get_browser('./chromedriver')

In [4]:
chrome.get('http://www.naver.com')

In [5]:
search_box = chrome.find_element_by_css_selector('#query')

In [6]:
search_box.send_keys('파이썬')

In [7]:
search_box.send_keys(web.Keys.RETURN)

In [8]:
chrome.quit()

### 네이버 검색 결과 링크 추출

크롬 브라우저에서 검색 결과 페이지 표시

In [9]:
chrome = web.get_browser('./chromedriver')
chrome.get('http://www.naver.com')

search_box = chrome.find_element_by_css_selector('#query')
search_box.send_keys('파이썬', web.Keys.RETURN)

검색 결과 링크 선택

In [10]:
search_results = chrome.find_elements_by_css_selector('.webdoc.section a.link')

In [11]:
search_results

[<selenium.webdriver.remote.webelement.WebElement (session="e2346f5f5bf48767596fb44de2754ddf", element="0.8373528349024337-1")>,
 <selenium.webdriver.remote.webelement.WebElement (session="e2346f5f5bf48767596fb44de2754ddf", element="0.8373528349024337-2")>,
 <selenium.webdriver.remote.webelement.WebElement (session="e2346f5f5bf48767596fb44de2754ddf", element="0.8373528349024337-3")>]

In [12]:
search_results[0].get_attribute('href')

'http://www.kyobobook.co.kr/product/detailViewKor.laf?barcode=9788968481819'

검색 결과 링크에서 URL과 설명 추출

In [13]:
네이버검색결과_URL= web.get_urls(search_results)

In [14]:
for 제목, url in 네이버검색결과_URL:
    print(제목, url)

파이썬 웹프로그래밍 http://www.kyobobook.co.kr/product/detailViewKor.laf?barcode=9788968481819
파이썬 생존 안내서 (자막) http://www.slideshare.net/sublee/ss-67589513
도도와 파이썬: 좋은 선택과 나쁜 선택 http://www.slideshare.net/shinvee/ss-52215845


크롬 브라우저 종료

In [15]:
chrome.quit()

추출한 검색 결과 링크를 엑셀 파일로 저장

In [16]:
from pandas import DataFrame
from pyautomate import office

In [17]:
네이버검색결과_URL표 = DataFrame(네이버검색결과_URL)

In [18]:
네이버검색결과_URL표

,text,url
0,파이썬 웹프로그래밍,http://www.kyobobook.co.kr/product/detailViewK...
1,파이썬 생존 안내서 (자막),http://www.slideshare.net/sublee/ss-67589513
2,도도와 파이썬: 좋은 선택과 나쁜 선택,http://www.slideshare.net/shinvee/ss-52215845


In [19]:
office.to_excel(네이버검색결과_URL표, '네이버 검색결과.xlsx')

# 비동기 로딩 웹페이지 정보 추출

종종 웹페이지는 페이지가 표시된 다음 계속해서 정보가 갱신될 수 있다.

In [20]:
html = web.parse_html('http://finance.naver.com/')
html.select(".num_quot > .num")

[<span class="num">2,065.77</span>,
 <span class="num">627.88</span>,
 <span class="num">266.52</span>]

In [21]:
import time

chrome =  web.get_browser('./chromedriver')
chrome.get('http://finance.naver.com/')
print('페이지 로딩 중 ...')
time.sleep(1)

증시정보 = []
for i in range(10):
    KOSPI지표_요소 = chrome.find_elements_by_css_selector(".num_quot > .num")
    시간표시_요소 = chrome.find_element_by_css_selector('#time')

    증시 = {}

    kospi = KOSPI지표_요소[0].text
    kospi = float(kospi.replace(',', ''))
    증시['KOSPI'] = kospi

    시간 = 시간표시_요소.text
    증시['시간'] = 시간
    
    증시정보.append(증시)

    print('{시간}\t KOSPI: {KOSPI}'.format(**증시))
    time.sleep(10)

chrome.quit()

페이지 로딩 중 ...
2017.01.16 13:49장중	 KOSPI: 2065.47
2017.01.16 13:49장중	 KOSPI: 2067.16
2017.01.16 13:49장중	 KOSPI: 2066.18
2017.01.16 13:49장중	 KOSPI: 2066.59
2017.01.16 13:49장중	 KOSPI: 2066.27
2017.01.16 13:49장중	 KOSPI: 2066.47
2017.01.16 13:50장중	 KOSPI: 2066.33
2017.01.16 13:50장중	 KOSPI: 2066.09
2017.01.16 13:50장중	 KOSPI: 2066.54
2017.01.16 13:50장중	 KOSPI: 2066.99


In [22]:
증시정보 = DataFrame(증시정보)

In [23]:
증시정보

,KOSPI,시간
0,2065.47,2017.01.16 13:49장중
1,2067.16,2017.01.16 13:49장중
2,2066.18,2017.01.16 13:49장중
3,2066.59,2017.01.16 13:49장중
4,2066.27,2017.01.16 13:49장중
5,2066.47,2017.01.16 13:49장중
6,2066.33,2017.01.16 13:50장중
7,2066.09,2017.01.16 13:50장중
8,2066.54,2017.01.16 13:50장중
9,2066.99,2017.01.16 13:50장중


In [24]:
office.to_excel(증시정보, 'KOSPI_20170116.xlsx')